Продолжаем знакомство с моделями МО в области обучения с учителем. На этот раз поговорим о задаче **классификации**. Вспомним, где находится классификация на нашей карте машинного обучения:

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@dst3-ml1-3_2 (1).png)

Вначале мы снова обратимся к классу линейных моделей и рассмотрим **логистическую регрессию**.

↓

Затем поговорим о **деревьях решений** для задачи классификации и научимся строить из этих деревьев целый лес.

**Цели данного модуля:**

* Познакомиться с принципами работы модели логистической регрессии для решения задачи классификации. 
* Рассмотреть метрики классификации и научиться оценивать качество моделей, решающих данную задачу.
* Узнать принципы построения деревьев решений.
* Изучить основы ансамблевых моделей типа бэггинг на примере случайного леса.
* Научиться применять деревья решений и случайные леса для решения задачи классификации.

`✍ Ранее мы обсуждали модель линейной регрессии, которая предназначена для решения задачи регрессии. Теперь нам предстоит разобраться с тем, как преобразовать данную модель, чтобы она решала задачу классификации.`

Для начала вспомним, что такое классификация.

**Задача классификации (classification)** *— задача, в которой мы пытаемся предсказать класс объекта на основе признаков в наборе данных. То есть задача сводится к предсказанию целевого признака, который является **категориальным**.*

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@dst3-ml1-3_9.png)

Когда классов, которые мы хотим предсказать, только два, классификация называется **бинарной**. Например, мы можем предсказать, болен ли пациент раком, является ли изображение человеческим лицом, является ли письмо спамом и т. д.

Когда классов, которые мы хотим предсказать, более двух, классификация называется **мультиклассовой (многоклассовой)**. Например, предсказание модели самолёта по радиолокационным снимкам, классификация животных на фотографиях, определение языка, на котором говорит пользователь, разделение писем на группы.

    → Для простоты мы пока разберёмся с бинарной классификацией, а в следующем юните обобщим результат на мультиклассовую.

Что вообще означает «решить задачу классификации»? Это значит построить разделяющую поверхность в пространстве признаков, которая делит пространство на части, каждая из которых соответствует определённому классу. 

Ниже представлены примеры разделяющих поверхностей, которые производят бинарную классификацию. Красным и синим цветом обозначены классы, зелёным — собственно поверхность, которая делит пространство признаков на две части. В каждой из этих частей находятся только наблюдения определённого класса.

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@dst3-ml3-2_1.png)

*Модели, которые решают задачу классификации, называются **классификаторами (classifier)**.*

Если взять в качестве разделяющей поверхности некоторую плоскость (ровная поверхность на первом рисунке), то мы получаем модель логистической регрессии, которая тесно связана с рассмотренной нами ранее линейной регрессией.

Давайте для начала вспомним, как выглядит уравнение модели линейной регрессии в общем случае:

![](./image/2024-08-01_20-59-15.png)

В общем случае это уравнение гиперплоскости, которая стремится приблизить зависимость целевой переменной от $m$ факторов.

* Когда фактор всего один, уравнение задаёт прямую:

$y^` = w_0 + w_1x$

* Когда факторов два, уравнение задаёт плоскость:

$y^`=w_0+w_1x_1+w_2x_2$

→ Но всё это работает только в том случае, когда целевой признак $y$, который мы хотим предсказать, является числовым, например цена, вес, время аренды и т. д.

Что же делать с этой моделью, когда целевой признак $y$ является категориальным? Например, является письмо спамом или обычным письмом?

Можно предположить, что, раз у нас есть две категории, мы можем обозначить категории за $y=1$ (Спам) и $y=0$ (Не спам) и обучить линейную регрессию предсказывать 0 и 1.

Но результат будет очень плохим. Выглядеть это будет примерно так:

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@dst3-ml3-2_2.png)

Для больших значений $x$ прямая будет выдавать значения больше 1, а для очень маленьких — меньше 0. Что это значит? Непонятно. Непонятно и то, что делать со значениями в диапазоне от 0 до 1. Да, можно относить значения на прямой выше 0.5 к классу 1, а меньше либо равным 0.5 — к классу 0, но это всё «костыли».

`Идея! Давайте переведём задачу классификации в задачу регрессии. Вместо предсказания класса будем предсказывать вероятность принадлежности к этому классу.` 

Модель должна выдавать некоторую вероятность $P$, которая будет определять, принадлежит ли данный объект к классу 1: например, вероятность того, что письмо является спамом. При этом вероятность того, что письмо является обычным письмом (класс 0), определяется как $Q=1-P$.  

Когда модель будет обучена на предсказание вероятности, мы зададим некоторый порог вероятности. Если предсказанная вероятность будет выше этого порога, мы определим объект к классу 1, а если ниже — к классу 0.

Например, стандартный порог равен 0.5. То есть если вероятность $P>0.5$, мы будем считать письмо спамом, а если $P<=0.5$ — обычным информативным письмом.

В итоге мы добьёмся того, что будем предсказывать не дискретный категориальный, а непрерывный числовой признак, который лежит в диапазоне [0, 1]. А это уже знакомая нам задача регрессии.

    → Однако остался главный вопрос: как научить модель предсказывать вероятности, ведь они должны лежать строго в диапазоне от 0 до 1, а предсказания линейной регрессии лежат в диапазоне от $-infinity$ до $+infinity$? 

Тут-то мы и приходим к модели логистической регрессии — **регрессии вероятностей**.

### <center> ОБЩЕЕ ПРЕДСТАВЛЕНИЕ О ЛОГИСТИЧЕСКОЙ РЕГРЕССИИ

**Логистическая регрессия (Logistic Regression)** *— одна из простейших моделей для решения задачи классификации. Несмотря на простоту, модель входит в топ часто используемых алгоритмов классификации в Data Science.*

В основе логистической регрессии лежит логистическая функция (*logistic function*) $sigma(z)$ — отсюда и название модели. Однако более распространённое название этой функции — **сигмόида (sigmoid)**. Записывается она следующим образом:

![](./image/2024-08-01_21-07-45.png)

Примечание. Здесь $e$ — экспонента или число Эйлера. Это число является бесконечным, а его значение обычно принимают равным $2.718...$.

А вот график зависимости сигмоиды от аргумента $z$:

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@dst3-ml3-2_3.png)

**В чём преимущество этой функции?**

*У сигмоиды есть два очень важных для нас свойства:*

* Значения сигмоиды $sigma(z)$ лежат в диапазоне от 0 до 1 при любых значения аргумента $z$: какой бы $z$ вы ни подставили, число меньше 0 или больше 1 вы не получите.
* Сигмоида выдаёт значения $sigma(z)>0.5$ при её аргументе $x>0$, $sigma(z)<0.5$ — при $z<0 и $sigma(z)=0.5$ — при $z=0$.

Это ведь и есть свойства вероятности! Выходом сигмоиды является число от 0 до 1, которое можно интерпретировать как вероятность принадлежности к классу 1. Её мы и пытаемся предсказать.

**Основная идея** модели логистической регрессии: возьмём модель линейной регрессии (обозначим её выход за $z$)

![](./image/2024-08-01_21-17-21.png)

и подставим выход модели $z$ в функцию сигмоиды, чтобы получить искомые оценки вероятности (в математике принято писать оценочные величины с «шапкой» наверху, а истинные значения — без «шапки», это чистая формальность):

![](./image/2024-08-01_21-18-54.png)

**Примечание.** *Далее в модуле мы будем называть оценки вероятности $P^`$ просто вероятностью, но только для краткости. Это не значит, что эти оценки являются истинными вероятностями принадлежности к каждому из классов (их нельзя сосчитать, так как для этого нужна выборка бесконечного объёма). Если вы употребляете термин «вероятности» на собеседованиях, обязательно предварительно укажите, что вы подразумеваете оценку вероятности.*

Обучать будем всё в совокупности, пытаясь получить наилучшую оценку вероятности $P^`$. Если вероятность $P^`>0.5$, относим объект к классу 1, а если $P^`<=0.5$, относим объект к классу 0. 

Математически это записывается следующей формулой:

![](./image/2024-08-01_21-21-56.png)

**Примечание.** *В данном выражении $I[P^`]$ называется **индикаторной функцией**. Она возвращает 1, если её значение больше какого-то порога, и 0 — в противном случае. Математики часто записывают просто квадратные скобки, опуская символ $I$: $[P^`]$.*

**Чего мы добились таким преобразованием?**

Если мы обучим модель, то есть подберём  коэффициенты $w_0, w_1, w_2, ..., w_m$ (как их найти, обсудим чуть позже) таким образом, что для объектов класса 1 модель линейной регрессии начнёт выдавать положительное число, а для класса 0 — выдавать отрицательное число, то тогда, подставив предсказание линейной регрессии $z$ в сигмоиду, мы сможем получать вероятности принадлежности к каждому из классов в диапазоне от 0 до 1.

Далее по порогу вероятности мы сможем определять, к какому классу принадлежит объект.

Это и есть наша цель. Мы свели задачу классификации к задаче регрессии для предсказания вероятностей. 

Для бинарной классификации описанное выше будет выглядеть следующим образом:

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@dst3-ml3-2_4.png)

![](./image/2024-08-01_21-26-21.png)

**Разберёмся с геометрией**

Возьмём частный случай, когда класс объекта зависит от двух признаков — $x_1$ и $x_2$.

![](./image/2024-08-01_21-31-25.png)

![](./image/2024-08-01_21-32-32.png)

**В чём математический секрет?**

Математически подстановка в уравнение плоскости точки, которая не принадлежит ей (находится ниже или выше), означает вычисление расстояния от этой точки до плоскости.

* Если точка находится ниже плоскости, расстояние будет отрицательным ($z<0$).
* Если точка находится выше плоскости, расстояние будет положительным ($z>0$).
* Если точка находится на самой плоскости, $z=0$.

Мы знаем, что подстановка отрицательных чисел в сигмоиду приведёт к вероятности $P^`<0.5$, а постановка положительных — к вероятности $P^`>0.5$. 

*Таким образом, ключевым моментом в предсказании логистической регрессии является расстояние от точки до разделяющей плоскости в пространстве факторов. Это расстояние в литературе часто называется* **отступом (margin)**. 

В этом и состоит секрет работы логистической регрессии.

*Чем больше расстояние от точки, находящейся выше разделяющей плоскости, до самой плоскости, тем больше оценка вероятности принадлежности к классу 1.*

Попробуйте подставить различные координаты точек в модель логистической регрессии и убедитесь в этом.

Можно построить тепловую карту, которая показывает, чему равны вероятности в каждой точке пространства:

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@dst3-ml3-2_7.png)

`На рисунке точки, которые относятся к классу непоступивших абитуриентов, лежащие ниже разделяющей плоскости, находятся в красной зоне. Чем насыщеннее красный цвет, тем ниже вероятность того, что абитуриент поступит в аспирантуру. И наоборот, точки, которые относятся к классу поступивших абитуриентов, лежащие выше разделяющей плоскости, находятся в синей зоне. Чем насыщеннее синий цвет, тем выше вероятность того, что абитуриент поступит в аспирантуру.`

Для случая зависимости целевого признака $y$ от трёх факторов $x_1$, $x_2$ и $x_3$, например от баллов за два экзамена и рейтинга университета, из которого выпустился абитуриент, выражение для $z$ будет иметь вид:

$z=w_0+w_1x_1+w_2x_2+w_3x_3$

Уравнение задаёт плоскость в четырёхмерном пространстве. Но если вспомнить, что $y$ — категориальный признак и классы можно обозначить цветом, то получится перейти в трёхмерное пространство. Разделяющая плоскость будет выглядеть следующим образом:

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@dst3-ml3-2_8.png)

В общем случае, когда у нас есть зависимость от $m$ факторов, линейное выражение, находящееся под сигмоидой, будет обозначать **разделяющую гиперплоскость**.

![](./image/2024-08-01_21-39-41.png)

### <center> ПОИСК ПАРАМЕТРОВ ЛОГИСТИЧЕСКОЙ РЕГРЕССИИ

Итак, мы разобрались с тем, как выглядит модель логистической регрессии и что она означает в геометрическом смысле.

Но остался один главный вопрос: как найти такие коэффициенты $w=(w_0, w_1, w_2, ..., w_m)$, чтобы гиперплоскость разделяла пространство наилучшим образом?

Вновь обратимся к нашей схеме минимизации эмпирического риска:

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@dst3-ml1-2_1.png)

Можно предположить, что стоит использовать метод наименьших квадратов. Введём функцию ошибки — средний квадрат разности MSE между истинными классами $y$ и предсказанными классами $y^`$ и попытаемся его минимизировать.

Сразу можно достоверно предсказать, что результат такого решения будет плохим, поэтому воздержимся от его использования.

Здесь нужен другой подход. Это **метод максимального правдоподобия (Maximum Likelihood Estimation — MLE)**. 

**Правдоподобие** *— это оценка того, насколько вероятно получить истинное значение целевой переменной  при данных  и параметрах $w$.*

Данный метод позволяет получить функцию правдоподобия.

**Цель метода** *— найти такие параметры $w=(w_0, w_1, w_2, ..., w_m)$, в которых наблюдается максимум функции правдоподобия.*

А мы пока что опустим математические детали метода и приведём только конечную формулу:

![](./image/2024-08-01_21-45-58.png)

Не пугайтесь. Давайте разберёмся, что есть что и как работает эта функция.

* $n$ — количество наблюдений.
* $y_i$ — это истинный класс (1 или 0) для -ого объекта из набора данных.
* $P^`=sigma(z_i)$ — предсказанная с помощью логистической регрессии вероятность принадлежности к классу 1 для $i$-ого объекта из набора данных.
* $z_i$ — результат подстановки -ого объекта из набора данных в уравнение разделяющей плоскости $z_i=w^`*x_i^`$.
* $log$ — логарифм (обычно используется натуральный логарифм по основанию $e-ln$).

![](./image/2024-08-01_21-54-39.png)

Такие расчёты можно производить для любых значений параметров, меняется только оценка вероятности $P_i^`$.

**Примечание.** *К сожалению, функция likelihood не имеет интерпретации, то есть нельзя сказать, что значит число -2.34 в контексте правдоподобия.*

Цель — найти такие параметры, при которых наблюдается максимум этой функции.

Теперь пора снова применить магию математики, чтобы привести задачу к привычному нам формату минимизации эмпирического риска. По правилам оптимизации, если поставить перед функцией минус, то задача оптимизации меняется на противоположную: был поиск максимума — станет поиском минимума.

Таким образом мы получим функцию потерь $L(w)$, которая носит название **«функция логистических потерь»**, или **logloss**. Также часто можно встретить название **кросс-энтропия**, или **cross-entropy loss**:

![](./image/2024-08-01_21-56-49.png)

Вот эту функцию мы и будем минимизировать в рамках поиска параметров логистической регрессии. Мы должны найти такие параметры разделяющей плоскости $w$, при которых наблюдается минимум logloss.

Знакомая задача? Всё то же самое, что и с линейной регрессией, только функция ошибки другая.

    → К сожалению, для такой функции потерь аналитическое решение оптимизационной задачи найти не получится: при расчётах получается, что его попросту не существует.

Но мы помним, что, помимо аналитических решений, есть и численные.

Например, для поиска параметров можно использовать знакомый нам градиентный спуск. Вспомним, как выглядит итерационная формула данного метода:

![](./image/2024-08-01_21-58-06.png)

Повторим её смысл: новое значение параметров $w^{(k+1)}$ получается путём сдвига текущих $w^{(k)}$ в сторону вектора антиградиента $Grad L(w^{(k)})$, умноженного на темп обучения $nu$.

    Мы уже знаем, что для того, чтобы повысить шанс пройти мимо локальных минимумов функции потерь, используется не сам градиентный спуск, а его модификации: например, можно использовать уже знакомый нам стохастический градиентный спуск (SGD).

`Помним, что применение градиентного спуска требует предварительного масштабирования данных (стандартизации/нормализации). В реализации логистической регрессии в sklearn предусмотрено ещё несколько методов оптимизации, для которых масштабирование не обязательно. О них мы упомянем в практической части модуля.`

Во избежание переобучения модели в функцию потерь логистической регрессии традиционно добавляется **регуляризация**. В реализации логистической регрессии в *sklearn* она немного отличается от той, что мы видели ранее для линейной регрессии.

При L1-регуляризации мы добавляем в функцию потерь $L(w)$ штраф из суммы модулей параметров, а саму функцию logloss умножаем на коэффициент $C$:

![](./image/2024-08-01_22-10-48.png)

Значение коэффициента $C$ — коэффициент, обратный коэффициенту регуляризации. Чем **больше** $C$, тем **меньше** «сила» регуляризации.

Предлагаем вам посмотреть на то, как будет меняться форма сигмоиды, разделяющей плоскости при минимизации функции потерь logloss (она обозначена как cross-entropy в виде концентрических кругов — вид сверху), с помощью обычного градиентного спуска (не стохастического) в виде анимации.

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@dst3-ml3-2_12.gif)
![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@dst3-ml3-2_13.gif)
![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@dst3-ml3-2_14.gif)

Не волнуйтесь, все громоздкие формулы уже реализованы в классических библиотеках, таких как sklearn. Но нам важно понимать принцип того, что происходит «под капотом», чтобы верно интерпретировать результат и по возможности управлять им.

### <center> ЛОГИСТИЧЕСКАЯ РЕГРЕССИЯ В SKLEARN

    Мы будем работать со знакомым вам по модулю «Очистка данных» набором данных о диабете, первоначально полученным в Национальном институте диабета, болезней органов пищеварения и почек.

    Наша цель будет состоять в том, чтобы диагностически предсказать, есть ли у пациента диабет. На выбор экземпляров из более крупной базы данных было наложено несколько ограничений. В частности, все пациенты здесь — женщины не моложе 21 года из индейского племени Пима.

В модуле по очистке данных вы уже производили очистку этого набора данных:

* удалили дубликаты,
* удалили неинформативный признак Gender,
* обработали «скрытые» пропуски в данных,
* избавились от потенциальных выбросов.

#### Результат представлен в ./Логистическая_регрессия.ipynb

    ✍ Мы научились обучать модель логистической регрессии, чтобы решать с её помощью задачу классификации. Теперь мы познакомимся с метриками классификации, чтобы научиться оценивать качество модели.

Однако прежде чем мы перейдём к этим метрикам, нам очень важно освежить в памяти некоторую терминологию из статистики, а именно вспомнить, что такое ошибки I и II рода.

### <center>ОШИБКИ I И II РОДА С ТОЧКИ ЗРЕНИЯ КЛАССИФИКАЦИИ

Давайте рассмотрим предсказания алгоритма $y_i^`$ на конкретном объекте $x_i$ под номером из данных $i$ с точки зрения статистических гипотез.

Будем считать класс 1 (диабет есть) положительным исходом (Positive), а класс 0 (диабета нет) — отрицательным (Negative).

**Примечание.** *На первый взгляд такая терминология может показаться не совсем этичной, так как наличие диабета — это всё-таки отрицательный сценарий для пациента. Однако для унификации терминологии в машинном обучении в большинстве задач объекты класса 1 считаются объектами с наличием некоторого эффекта (болезнь есть / задолженность погашена / клиент ушёл / устройство отказало и т. д.), а объекты класса 0 — объектами с отсутствием этого эффекта (болезни нет / задолженность не погашена / клиент не ушёл / устройство работает без отказов и т. д.).*

Пусть у нас есть некоторый пациент $x_i$, и мы хотим понять, болен ли он диабетом. С точки зрения задачи классификации мы хотим предсказать истинный класс ($y_i$) пациента.

Нулевая гипотеза будет состоять в отсутствии эффекта (пациент не болен диабетом), то есть , а альтернативная — в его наличии (пациент болен диабетом) $y_i=0$, то есть $y_i=1$. В терминах статистических гипотез это будет записано так:

* $H_0$: Пациент $x_i$ не болеет диабетом $y_i=0$.
* $H_1$: Пациент $x_i$ болеет диабетом $y_i=1$.
Тогда у нас есть два случая, в которых мы можем допустить ошибку:

Ошибка I (первого) рода ($alpfa$-ошибка): отклонение нулевой гипотезы, когда она на самом деле верна, или **ложноположительный результат**. То есть мы предсказали, что пациент болен диабетом, хотя это не так.
Ошибка II (второго) рода ($betta$-ошибка): принятие нулевой гипотезы, когда она на самом деле ложна, или **ложноотрицательный результат**. То есть мы предсказали, что пациент здоров, хотя на самом деле он болен диабетом.

**Примечание.** *Как вы можете понять, в диагностических задачах для нас критичнее ошибка II рода. Последствия будут более серьёзными, если мы примем больного пациента за здорового, чем если мы примем здорового за больного. Нам важно охватить всех потенциально больных пациентов, чтобы сделать дополнительный анализ и удостовериться в результате.*

    Отлично, мы освежили в памяти ошибки I и II рода — это поможет нам понять суть метрик классификации. Давайте перейдём к ним.